In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

In [2]:
import numpy as np

In [3]:
from sklearn.metrics import make_scorer

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import category_encoders

In [5]:
from sklearn.model_selection import GridSearchCV

In [6]:
import pandas as pd

In [7]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [8]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [9]:
processed = pd.read_csv('features_01.csv')

In [10]:
processed = processed.set_index('person')
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
person,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid


In [11]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [12]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
person,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,Referral
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,Paid
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,Paid
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,Paid
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,Ninguno


In [21]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [22]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,0.066079
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,0.042803
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,0.042803
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,0.042803
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,0.017361


In [23]:
normalized = normalize(encoded_training,axis=0)

In [28]:
params={
    'weights':['uniform','distance'],
    'leaf_size':[15,30,45],
    'p':[1,2,3]
}
grid = GridSearchCV(KNeighborsClassifier(),params,scoring='roc_auc',cv=2,verbose=3)

In [29]:
xtr,xte,ytr,yte=train_test_split(normalized,training_labels)

In [30]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] leaf_size=15, p=1, weights=uniform ..............................
[CV]  leaf_size=15, p=1, weights=uniform, score=0.5917783227261586, total=   0.4s
[CV] leaf_size=15, p=1, weights=uniform ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  leaf_size=15, p=1, weights=uniform, score=0.5863252166612535, total=   0.5s
[CV] leaf_size=15, p=1, weights=distance .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV]  leaf_size=15, p=1, weights=distance, score=0.5934491891981089, total=   0.4s
[CV] leaf_size=15, p=1, weights=distance .............................
[CV]  leaf_size=15, p=1, weights=distance, score=0.5860628042947084, total=   0.4s
[CV] leaf_size=15, p=2, weights=uniform ..............................
[CV]  leaf_size=15, p=2, weights=uniform, score=0.5995142553908234, total=   0.4s
[CV] leaf_size=15, p=2, weights=uniform ..............................
[CV]  leaf_size=15, p=2, weights=uniform, score=0.5909377579600095, total=   0.5s
[CV] leaf_size=15, p=2, weights=distance .............................
[CV]  leaf_size=15, p=2, weights=distance, score=0.5995255246335585, total=   0.4s
[CV] leaf_size=15, p=2, weights=distance .............................
[CV]  leaf_size=15, p=2, weights=distance, score=0.5915412661557675, total=   0.5s
[CV] leaf_size=15, p=3, weights=uniform ..............................
[CV]  leaf_size=15, p=3, weights=uniform, score=0.6023388105877755, total=   2

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  1.9min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'weights': ['uniform', 'distance'], 'leaf_size': [15, 30, 45], 'p': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [31]:
grid.best_params_

{'leaf_size': 15, 'p': 3, 'weights': 'distance'}

In [32]:
grid.score(X=xte,y=yte)

0.5947141726733516

In [33]:
predictor = grid.best_estimator_

In [34]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [35]:
predict_features['label'] = prediction_submit

In [36]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [37]:
subm = subm.set_index('person')